# 🌡️ Limiting Factors
## Understanding What Controls Life in Ecosystems

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/The-Pattern-Hunter/interactive-ecology-biometry/blob/main/unit-1-ecosystem/notebooks/05_limiting_factors.ipynb)

---

> *"Life is not limited by total resources available, but by the scarcest resource."* - Justus von Liebig

### 🎯 Learning Objectives

By the end of this notebook, you will:
1. Understand **Liebig's Law of the Minimum**
2. Apply **Shelford's Law of Tolerance**
3. Identify **optimal ranges** vs **tolerance limits**
4. Analyze how **physical factors** limit organisms
5. Explore **temperature, light, water, and nutrient** effects
6. Understand the **Q₁₀ rule** for temperature
7. Connect limiting factors to **ecosystem distribution**

In [ ]:
# Setup
!pip install numpy pandas plotly matplotlib scipy -q

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats

print("✅ Ready to explore limiting factors!")
print("🌡️ Let's discover what controls life!")

---

## 📊 Part 1: Liebig's Law of the Minimum

### The Barrel Analogy 🛢️

**Imagine a barrel made of staves (wooden planks) of different heights:**

```
    |     |         |     |
    |     |    |    |     |
    |     |    |    |     |
    |  |  |    |    |     |
    |  |  |    |    |  |  |
    |__|__|____|____|__|__|
       ↑
   SHORTEST STAVE
   (Limiting Factor)
```

**The barrel can only hold water up to the HEIGHT OF THE SHORTEST STAVE!**

### Liebig's Law (1840):

**"Growth is controlled not by the total resources available, but by the scarcest resource (limiting factor)."**

### In Biology:

Even if a plant has:
- ✅ Plenty of sunlight
- ✅ Abundant water
- ✅ Perfect temperature
- ✅ Good soil
- ❌ **But NO nitrogen**

**→ The plant CANNOT grow!** (Nitrogen is the limiting factor)

### Examples:

| Ecosystem | Often Limited By |
|-----------|------------------|
| **Desert** | Water 💧 |
| **Arctic Tundra** | Temperature 🌡️ |
| **Deep Ocean** | Light ☀️ |
| **Tropical Rainforest** | Phosphorus 🪨 |
| **Open Ocean** | Iron Fe |
| **Temperate Forest** | Nitrogen 💨 |

In [ ]:
# Interactive Liebig's Barrel
def create_liebig_barrel(nutrient_levels):
    """
    Create Liebig's barrel visualization
    nutrient_levels: dict with nutrient names and their availability (0-100)
    """
    nutrients = list(nutrient_levels.keys())
    levels = list(nutrient_levels.values())
    
    # Find limiting factor (minimum)
    limiting_idx = np.argmin(levels)
    limiting_nutrient = nutrients[limiting_idx]
    limiting_level = levels[limiting_idx]
    
    # Create bar chart
    colors = ['red' if i == limiting_idx else 'lightblue' for i in range(len(nutrients))]
    
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=nutrients,
        y=levels,
        marker_color=colors,
        text=[f"{l}%" for l in levels],
        textposition='outside',
        showlegend=False
    ))
    
    # Add line showing growth limit
    fig.add_hline(
        y=limiting_level,
        line_dash="dash",
        line_color="red",
        line_width=3,
        annotation_text=f"Max Growth Limited to {limiting_level}%"
    )
    
    fig.update_layout(
        title=f"🛢️ Liebig's Barrel: {limiting_nutrient} is LIMITING<br><sub>Red bar = Shortest stave</sub>",
        xaxis_title="Nutrient",
        yaxis_title="Availability (%)",
        height=500,
        template='plotly_white',
        yaxis=dict(range=[0, 110])
    )
    
    return fig, limiting_nutrient, limiting_level

# Example 1: Nitrogen limiting
scenario1 = {
    'Water': 90,
    'Light': 85,
    'Temperature': 80,
    'Nitrogen': 20,  # LIMITING!
    'Phosphorus': 75,
    'Potassium': 70
}

fig1, limiting1, level1 = create_liebig_barrel(scenario1)
fig1.show()

print(f"\n📊 Analysis:")
print(f"   Limiting Factor: {limiting1}")
print(f"   Maximum Growth: {level1}% of potential")
print(f"\n💡 Solution:")
print(f"   Adding {limiting1} will increase growth")
print(f"   Adding other nutrients will NOT help!")

In [ ]:
# Interactive: Compare different scenarios
scenarios = {
    'Desert': {
        'Water': 10,      # LIMITING!
        'Light': 100,
        'Temperature': 90,
        'Nitrogen': 60,
        'Phosphorus': 70
    },
    'Arctic': {
        'Water': 80,
        'Light': 40,
        'Temperature': 15,  # LIMITING!
        'Nitrogen': 50,
        'Phosphorus': 60
    },
    'Tropical Rainforest': {
        'Water': 95,
        'Light': 90,
        'Temperature': 95,
        'Nitrogen': 70,
        'Phosphorus': 25   # LIMITING!
    }
}

print("🌍 Limiting Factors in Different Ecosystems:\n")

for ecosystem, nutrients in scenarios.items():
    limiting = min(nutrients, key=nutrients.get)
    level = nutrients[limiting]
    print(f"   {ecosystem}:")
    print(f"      Limiting Factor: {limiting} ({level}%)")
    print(f"      This explains why {ecosystem.lower()} has characteristic vegetation!\n")

---

## 📈 Part 2: Shelford's Law of Tolerance

### Beyond the Minimum:

Liebig said: **"Too little is bad"**  
Shelford added: **"Too much is ALSO bad!"** (1911)

### The Tolerance Curve 📊

Every organism has:
- **Optimum range**: Best performance
- **Range of tolerance**: Can survive
- **Lower limit**: Too little → death
- **Upper limit**: Too much → death

```
        PERFORMANCE
             ^
             |     /‾‾‾‾‾\
  Optimum    |    /       \
             |   /         \
  Stress     |  /           \
             | /             \
  Death      |/_______________\___
             |                    → FACTOR
            Min    Opt    Max
```

### Key Terms:

- **Zone of Intolerance**: Organism dies
- **Zone of Physiological Stress**: Survives but struggles
- **Zone of Optimal Range**: Thrives
- **Eurythermal**: Wide tolerance (e.g., cockroach)
- **Stenothermal**: Narrow tolerance (e.g., coral)

In [ ]:
# Create interactive tolerance curve
def create_tolerance_curve(factor_name, min_val, opt_low, opt_high, max_val, species_name):
    """
    Create tolerance curve for an environmental factor
    """
    # Create x values
    x = np.linspace(min_val - 5, max_val + 5, 1000)
    
    # Create bell curve centered on optimal range
    opt_center = (opt_low + opt_high) / 2
    opt_width = (opt_high - opt_low) / 2
    
    # Gaussian-like curve
    sigma = (max_val - min_val) / 6
    y = np.exp(-0.5 * ((x - opt_center) / sigma) ** 2)
    
    # Set to zero outside limits
    y[x < min_val] = 0
    y[x > max_val] = 0
    
    fig = go.Figure()
    
    # Main curve
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='lines',
        line=dict(color='blue', width=3),
        fill='tozeroy',
        fillcolor='rgba(0, 0, 255, 0.3)',
        name='Performance'
    ))
    
    # Mark zones
    fig.add_vrect(
        x0=opt_low, x1=opt_high,
        fillcolor='green', opacity=0.2,
        annotation_text="Optimal Range",
        annotation_position="top left"
    )
    
    fig.add_vrect(
        x0=min_val, x1=opt_low,
        fillcolor='yellow', opacity=0.2,
        annotation_text="Stress Zone",
        annotation_position="top left"
    )
    
    fig.add_vrect(
        x0=opt_high, x1=max_val,
        fillcolor='yellow', opacity=0.2,
        annotation_text="Stress Zone",
        annotation_position="top right"
    )
    
    # Mark critical points
    fig.add_vline(x=min_val, line_dash="dash", line_color="red",
                  annotation_text=f"Lower Limit ({min_val})")
    fig.add_vline(x=max_val, line_dash="dash", line_color="red",
                  annotation_text=f"Upper Limit ({max_val})")
    
    fig.update_layout(
        title=f"📈 Tolerance Curve: {species_name}<br><sub>{factor_name}</sub>",
        xaxis_title=factor_name,
        yaxis_title="Performance (Relative)",
        height=500,
        template='plotly_white',
        showlegend=False
    )
    
    return fig

# Example: Temperature tolerance of a temperate plant
fig = create_tolerance_curve(
    factor_name="Temperature (°C)",
    min_val=5,
    opt_low=15,
    opt_high=25,
    max_val=35,
    species_name="Temperate Oak Tree"
)
fig.show()

print("\n🌡️ Tolerance Zones:")
print("   • Below 5°C: Death (too cold)")
print("   • 5-15°C: Stress zone (survives but struggles)")
print("   • 15-25°C: Optimal range (thrives)")
print("   • 25-35°C: Stress zone (heat stress)")
print("   • Above 35°C: Death (too hot)")

---

## 🌡️ Part 3: Temperature as a Limiting Factor

### Why Temperature Matters:

Temperature affects:
- ⚡ **Metabolic rate** (speed of chemical reactions)
- 🧬 **Enzyme function**
- 💧 **Water availability**
- 🍃 **Photosynthesis rate**
- 🦋 **Development time**

### The Q₁₀ Rule:

**Q₁₀** = Temperature coefficient

**Definition**: Factor by which reaction rate increases for every 10°C rise in temperature

**Formula**:
```
Q₁₀ = (Rate at T+10°C) / (Rate at T°C)
```

**Typical values**:
- Chemical reactions: Q₁₀ ≈ 2-3
- Biological processes: Q₁₀ ≈ 2
- Example: If Q₁₀ = 2, then:
  - 20°C → Rate = 1x
  - 30°C → Rate = 2x (doubled!)
  - 40°C → Rate = 4x

In [ ]:
# Interactive Q₁₀ demonstration
def calculate_q10_effect(base_temp, base_rate, q10_value, temp_range):
    """
    Calculate how rate changes with temperature using Q₁₀
    """
    temps = np.arange(temp_range[0], temp_range[1] + 1, 1)
    rates = base_rate * (q10_value ** ((temps - base_temp) / 10))
    return temps, rates

# Compare different Q₁₀ values
fig = go.Figure()

base_temp = 20
base_rate = 10
temp_range = (0, 50)

q10_values = [1.5, 2.0, 2.5, 3.0]
colors_q10 = ['blue', 'green', 'orange', 'red']

for q10, color in zip(q10_values, colors_q10):
    temps, rates = calculate_q10_effect(base_temp, base_rate, q10, temp_range)
    
    fig.add_trace(go.Scatter(
        x=temps,
        y=rates,
        mode='lines',
        line=dict(color=color, width=3),
        name=f'Q₁₀ = {q10}'
    ))

fig.update_layout(
    title=f"🌡️ The Q₁₀ Effect on Metabolic Rate<br><sub>Base: {base_rate} units at {base_temp}°C</sub>",
    xaxis_title="Temperature (°C)",
    yaxis_title="Metabolic Rate (Relative)",
    height=500,
    template='plotly_white'
)

fig.show()

print("\n🔥 Q₁₀ Interpretation:")
print(f"   Starting at {base_temp}°C with rate = {base_rate}")
print(f"\n   If Q₁₀ = 2.0:")
temps_example, rates_example = calculate_q10_effect(base_temp, base_rate, 2.0, temp_range)
print(f"      At 20°C: Rate = {base_rate:.0f}")
print(f"      At 30°C: Rate = {base_rate * 2:.0f} (2x)")
print(f"      At 40°C: Rate = {base_rate * 4:.0f} (4x)")
print(f"\n💡 Higher Q₁₀ = More sensitive to temperature!")

In [ ]:
# Compare temperature tolerance of different organisms
organisms = {
    'Arctic Fox': {'min': -50, 'opt_low': -20, 'opt_high': 10, 'max': 25, 'type': 'Stenothermal (cold)'},
    'Camel': {'min': 0, 'opt_low': 20, 'opt_high': 45, 'max': 55, 'type': 'Eurythermal'},
    'Coral Reef': {'min': 18, 'opt_low': 25, 'opt_high': 29, 'max': 32, 'type': 'Stenothermal (warm)'},
    'Cockroach': {'min': 0, 'opt_low': 15, 'opt_high': 35, 'max': 45, 'type': 'Eurythermal'}
}

fig = go.Figure()

for i, (name, data) in enumerate(organisms.items()):
    # Create tolerance range
    fig.add_trace(go.Scatter(
        x=[data['min'], data['opt_low'], data['opt_high'], data['max']],
        y=[i, i, i, i],
        mode='lines+markers',
        line=dict(width=10),
        marker=dict(size=15),
        name=f"{name} ({data['type']})",
        hovertemplate=f"<b>{name}</b><br>" +
                     f"Min: {data['min']}°C<br>" +
                     f"Optimal: {data['opt_low']}-{data['opt_high']}°C<br>" +
                     f"Max: {data['max']}°C<extra></extra>"
    ))

fig.update_layout(
    title="🌡️ Temperature Tolerance: Stenothermal vs Eurythermal<br><sub>Wider line = Broader tolerance</sub>",
    xaxis_title="Temperature (°C)",
    yaxis=dict(
        ticktext=list(organisms.keys()),
        tickvals=list(range(len(organisms)))
    ),
    height=500,
    template='plotly_white'
)

fig.show()

print("\n🌡️ Tolerance Comparison:")
print("   Stenothermal (Narrow tolerance):")
print("      • Arctic Fox: Cold specialist")
print("      • Coral Reef: Warm specialist")
print("      • Very sensitive to temperature change!")
print("\n   Eurythermal (Broad tolerance):")
print("      • Camel: Wide temperature range")
print("      • Cockroach: Adaptable")
print("      • Can survive in many climates")

---

## ☀️ Part 4: Light as a Limiting Factor

### Why Light Matters:

- 🌱 **Photosynthesis**: Energy source for producers
- 🌳 **Plant growth**: Determines height, leaf area
- 🦋 **Behavior**: Photoperiod affects reproduction, migration
- 🏞️ **Ecosystem structure**: Vertical stratification in forests

### Light Intensity Effects:

**Too little light**:
- Cannot photosynthesize enough
- Slow growth
- Eventually death

**Optimal light**:
- Maximum photosynthesis
- Healthy growth

**Too much light**:
- Photoinhibition (damage to photosystem)
- Water stress (high evaporation)
- Bleaching

### Plant Adaptations:

| Type | Light Preference | Adaptations | Examples |
|------|------------------|-------------|----------|
| **Sun plants** | High light | Thick leaves, high photosynthetic capacity | Corn, sunflower |
| **Shade plants** | Low light | Thin leaves, large surface area, chlorophyll-rich | Ferns, understory plants |

In [ ]:
# Light response curves for sun vs shade plants
light_intensity = np.linspace(0, 2000, 100)  # μmol photons/m²/s

# Sun plant (saturation at high light)
sun_max = 25
sun_response = sun_max * (1 - np.exp(-0.003 * light_intensity))

# Shade plant (saturation at low light, inhibition at high)
shade_max = 15
shade_response = shade_max * (1 - np.exp(-0.01 * light_intensity)) * np.exp(-0.0005 * light_intensity)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=light_intensity,
    y=sun_response,
    mode='lines',
    line=dict(color='orange', width=3),
    name='Sun Plant (e.g., Corn)'
))

fig.add_trace(go.Scatter(
    x=light_intensity,
    y=shade_response,
    mode='lines',
    line=dict(color='green', width=3),
    name='Shade Plant (e.g., Fern)'
))

# Mark typical forest floor light (1-2% of full sun)
fig.add_vline(x=40, line_dash="dash", line_color="brown",
              annotation_text="Forest Floor (~2% sun)")

# Mark full sunlight
fig.add_vline(x=2000, line_dash="dash", line_color="yellow",
              annotation_text="Full Sunlight")

fig.update_layout(
    title="☀️ Light Response Curves: Sun vs Shade Plants<br><sub>Photosynthesis rate vs light intensity</sub>",
    xaxis_title="Light Intensity (μmol photons/m²/s)",
    yaxis_title="Photosynthesis Rate (μmol CO₂/m²/s)",
    height=500,
    template='plotly_white'
)

fig.show()

print("\n☀️ Key Observations:")
print("   Sun Plants (Orange):")
print("      • Need high light to reach maximum photosynthesis")
print("      • Saturate at ~1500 μmol/m²/s")
print("      • Cannot survive in shade")
print("\n   Shade Plants (Green):")
print("      • Saturate at low light (~200 μmol/m²/s)")
print("      • Actually INHIBITED by high light!")
print("      • Adapted to forest understory")
print("\n   Forest Floor: Only 1-2% of canopy light reaches ground!")

---

## 💧 Part 5: Water as a Limiting Factor

### Why Water Matters:

- 🌊 **Universal solvent**: Transport medium
- 🌡️ **Temperature regulation**: Evaporative cooling
- 🌱 **Cell structure**: Turgor pressure
- 🍃 **Photosynthesis**: Raw material (6H₂O + 6CO₂ → ...)
- 💧 **Metabolic processes**: All reactions need water

### Water Availability Spectrum:

| Environment | Annual Rainfall | Plant Adaptations |
|-------------|----------------|-------------------|
| **Rainforest** | >2000 mm | Large leaves, shallow roots |
| **Temperate** | 500-1500 mm | Deciduous, seasonal growth |
| **Grassland** | 250-750 mm | Deep roots, C₄ photosynthesis |
| **Desert** | <250 mm | CAM photosynthesis, succulence, spines |

In [ ]:
# Water stress response curve
water_potential = np.linspace(-6, 0, 100)  # MPa (megapascals)

# Different plant types
mesophyte = 100 * np.exp(2 * water_potential)  # Temperate plants
xerophyte = 100 * np.exp(0.8 * water_potential)  # Desert plants
hydrophyte = 100 * (water_potential > -1.5).astype(float) * np.exp(5 * (water_potential + 1.5))  # Aquatic

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=water_potential,
    y=mesophyte,
    mode='lines',
    line=dict(color='green', width=3),
    name='Mesophyte (Temperate)',
    fill='tozeroy',
    fillcolor='rgba(0, 255, 0, 0.2)'
))

fig.add_trace(go.Scatter(
    x=water_potential,
    y=xerophyte,
    mode='lines',
    line=dict(color='brown', width=3),
    name='Xerophyte (Desert)',
    fill='tozeroy',
    fillcolor='rgba(165, 42, 42, 0.2)'
))

fig.add_trace(go.Scatter(
    x=water_potential,
    y=hydrophyte,
    mode='lines',
    line=dict(color='blue', width=3),
    name='Hydrophyte (Aquatic)',
    fill='tozeroy',
    fillcolor='rgba(0, 0, 255, 0.2)'
))

# Mark typical soil conditions
fig.add_vline(x=-0.5, line_dash="dot", line_color="green",
              annotation_text="Well-watered soil")
fig.add_vline(x=-1.5, line_dash="dot", line_color="orange",
              annotation_text="Wilting point")
fig.add_vline(x=-4, line_dash="dot", line_color="red",
              annotation_text="Desert soil")

fig.update_layout(
    title="💧 Water Stress Tolerance in Different Plant Types<br><sub>More negative = Drier conditions</sub>",
    xaxis_title="Soil Water Potential (MPa)",
    yaxis_title="Plant Performance (%)",
    height=500,
    template='plotly_white'
)

fig.show()

print("\n💧 Plant Water Strategies:")
print("   Hydrophytes (Blue):")
print("      • Cannot tolerate drought at all")
print("      • Die quickly when soil dries")
print("      • Examples: Water lilies, mangroves")
print("\n   Mesophytes (Green):")
print("      • Moderate tolerance")
print("      • Wilt around -1.5 MPa")
print("      • Examples: Most crop plants, temperate trees")
print("\n   Xerophytes (Brown):")
print("      • High drought tolerance")
print("      • Can survive down to -6 MPa or lower!")
print("      • Examples: Cacti, creosote bush")

---

## 🪨 Part 6: Soil Nutrients as Limiting Factors

### Essential Nutrients:

Plants need 17 essential elements:

#### Macronutrients (Large amounts):
- **C, H, O**: From air and water
- **N, P, K**: Often limiting (NPK fertilizers)
- **Ca, Mg, S**: Secondary macronutrients

#### Micronutrients (Trace amounts):
- **Fe, Mn, Zn, Cu, B, Mo, Cl, Ni**: Small but essential

### Most Common Limiting Nutrients:

1. **Nitrogen (N)**: Most often limiting on land
2. **Phosphorus (P)**: Often limiting in tropics
3. **Iron (Fe)**: Limiting in open ocean
4. **Potassium (K)**: Sometimes limiting

In [ ]:
# Fertilizer experiment demonstration
treatments = ['Control\n(No fertilizer)', 'N only', 'P only', 'K only', 'NPK\n(All three)']
yield_increase = [0, 50, 20, 15, 100]  # % increase over control
colors_fert = ['gray', 'lightblue', 'orange', 'yellow', 'green']

fig = go.Figure()

fig.add_trace(go.Bar(
    x=treatments,
    y=yield_increase,
    marker_color=colors_fert,
    text=[f"+{y}%" for y in yield_increase],
    textposition='outside'
))

fig.update_layout(
    title="🌾 Effect of Different Fertilizers on Crop Yield<br><sub>Nitrogen is the primary limiting factor</sub>",
    xaxis_title="Treatment",
    yaxis_title="Yield Increase (%)",
    height=500,
    template='plotly_white'
)

fig.show()

print("\n🌾 Fertilizer Experiment Results:")
print("   • Control: Baseline (0% increase)")
print("   • N only: +50% (N is MOST limiting!)")
print("   • P only: +20% (Secondary limitation)")
print("   • K only: +15% (Minor limitation)")
print("   • NPK all: +100% (All limitations removed!)")
print("\n💡 This demonstrates Liebig's Law:")
print("   Even with P and K, plants can't grow well without N")
print("   Must address the MOST limiting factor first!")

---

## 🌍 Part 7: Interactive Limiting Factor Simulator

In [ ]:
# Complete ecosystem simulator
def ecosystem_simulator(temperature, water, nitrogen, phosphorus, light):
    """
    Simulate plant growth based on multiple factors
    All inputs: 0-100 scale
    """
    factors = {
        'Temperature': temperature,
        'Water': water,
        'Nitrogen': nitrogen,
        'Phosphorus': phosphorus,
        'Light': light
    }
    
    # Find limiting factor
    limiting_factor = min(factors, key=factors.get)
    limiting_value = factors[limiting_factor]
    
    # Calculate growth (limited by minimum)
    growth = limiting_value
    
    # Create visualization
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Factor Availability', 'Plant Growth Prediction'),
        specs=[[{'type': 'bar'}, {'type': 'indicator'}]]
    )
    
    # Bar chart of factors
    factor_names = list(factors.keys())
    factor_values = list(factors.values())
    bar_colors = ['red' if v == limiting_value else 'lightblue' for v in factor_values]
    
    fig.add_trace(
        go.Bar(
            x=factor_names,
            y=factor_values,
            marker_color=bar_colors,
            showlegend=False
        ),
        row=1, col=1
    )
    
    # Add limiting line
    fig.add_hline(
        y=limiting_value,
        line_dash="dash",
        line_color="red",
        line_width=2,
        row=1, col=1
    )
    
    # Growth gauge
    fig.add_trace(
        go.Indicator(
            mode="gauge+number+delta",
            value=growth,
            title={'text': f"Growth Rate<br><sub>Limited by: {limiting_factor}</sub>"},
            delta={'reference': 100},
            gauge={
                'axis': {'range': [None, 100]},
                'bar': {'color': "darkgreen"},
                'steps': [
                    {'range': [0, 25], 'color': "red"},
                    {'range': [25, 50], 'color': "yellow"},
                    {'range': [50, 75], 'color': "lightgreen"},
                    {'range': [75, 100], 'color': "green"}
                ],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': limiting_value
                }
            }
        ),
        row=1, col=2
    )
    
    fig.update_xaxes(title_text="Environmental Factor", row=1, col=1)
    fig.update_yaxes(title_text="Availability (%)", row=1, col=1, range=[0, 110])
    
    fig.update_layout(
        title="🌱 Ecosystem Growth Simulator (Liebig's Law in Action)",
        height=500
    )
    
    return fig, limiting_factor, growth

# Example scenarios
print("🌍 Test Different Ecosystem Scenarios:\n")

scenarios_sim = {
    'Ideal Conditions': (85, 90, 80, 75, 95),
    'Desert': (80, 15, 60, 70, 100),  # Water limiting
    'Arctic': (10, 70, 50, 60, 40),   # Temperature limiting
    'Rainforest': (95, 100, 70, 25, 80)  # Phosphorus limiting
}

for scenario_name, (temp, water, N, P, light) in scenarios_sim.items():
    fig, limiting, growth = ecosystem_simulator(temp, water, N, P, light)
    print(f"{scenario_name}:")
    print(f"   Limiting Factor: {limiting}")
    print(f"   Growth Rate: {growth}%\n")

# Show one example
fig_example, _, _ = ecosystem_simulator(80, 15, 60, 70, 100)  # Desert
fig_example.show()

print("\n💡 Try changing the values to see how different factors limit growth!")

---

## 🎓 Summary

### Key Takeaways:

✅ **Liebig's Law**: Growth limited by scarcest resource (shortest stave)  
✅ **Shelford's Law**: Too little OR too much can be limiting  
✅ **Tolerance curves**: Optimal range, stress zones, death zones  
✅ **Temperature**: Q₁₀ rule, metabolic rates double per 10°C  
✅ **Light**: Sun vs shade plants, photoinhibition possible  
✅ **Water**: Critical for all life, determines ecosystem type  
✅ **Nutrients**: N and P most commonly limiting  
✅ **Adaptations**: Organisms evolve to tolerate local conditions  

### Real-World Applications:

#### 🌾 **Agriculture**:
- Identify limiting nutrients → Apply correct fertilizer
- Optimize irrigation
- Choose crops suited to local climate

#### 🌳 **Conservation**:
- Predict species distributions
- Understand climate change impacts
- Manage habitats effectively

#### 🏭 **Environmental Management**:
- Control eutrophication (limit P and N)
- Restore degraded ecosystems
- Predict invasive species spread

### The Big Picture:

**Every organism lives within constraints:**
- Too cold → Can't survive Arctic
- Too hot → Can't survive desert
- Too dry → Can't survive without water
- Too little N → Can't grow

**These constraints determine:**
- Where species can live (geographic range)
- How abundant they can be (population size)
- How ecosystems are structured (community composition)

### Integration with Other Concepts:

Limiting factors connect to:
- **Energy flow**: Determines productivity
- **Biogeochemical cycles**: Nutrient availability
- **Population ecology**: Carrying capacity
- **Climate change**: Shifting tolerance ranges

---

## 🎉 Unit 1 Complete!

### What You've Learned:

1. ✅ **Ecosystem Basics** - Autecology vs Synecology
2. ✅ **Food Webs & Energy** - 10% rule, trophic levels
3. ✅ **Ecological Pyramids** - Numbers, biomass, energy
4. ✅ **Biogeochemical Cycles** - C, N, P, Water
5. ✅ **Limiting Factors** - Liebig, Shelford, tolerance

### Next Steps:

**Continue your ecology journey:**
- **[Unit 2: Population Ecology](../../unit-2-population/)** - Growth, regulation, life history
- **[Unit 3: Community Ecology](../../unit-3-community/)** - Interactions, succession, diversity

**Apply your knowledge:**
- Design field experiments
- Analyze real ecosystem data
- Contribute to conservation

---

<div align="center">

**🎊 Congratulations on completing Unit 1! 🎊**

**Made with 💚 by Dr. Alok Patel**

[📓 Previous: Biogeochemical Cycles](04_biogeochemical_cycles.ipynb) | 
[🏠 Unit 1 Home](../../) | 
[📚 Main Repository](../../../)

</div>